<a href="https://colab.research.google.com/github/IyadKhuder/NLP_TensorFlowHub_Movie_Reviews/blob/main/NLP_TensorFlowHub_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Hub - Text classification

- Based on: https://www.tensorflow.org/hub/tutorials/tf2_text_classification

# Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print('TensorFlow version: ', tf.__version__)
print('TensorFlow Hub version: ', hub.__version__)

TensorFlow version:  2.9.2
TensorFlow Hub version:  0.12.0


# Dataset

- Extracted from imdb: 0 - negative, 1 - positive

In [2]:
train_data, test_data = tfds.load(name = 'imdb_reviews', split =['train', 'test'], batch_size=-1, as_supervised=True)
X_train, y_train = tfds.as_numpy(train_data)
X_test, y_test = tfds.as_numpy(test_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteO2011P/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteO2011P/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteO2011P/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
X_train.shape, y_train.shape

((25000,), (25000,))

In [4]:
X_test.shape, y_test.shape

((25000,), (25000,))

In [5]:
X_train[0:2]

array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot 

In [6]:
y_train[0:2]

array([0, 0])

In [7]:
X_test[0:2]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [8]:
y_test[0:2]

array([1, 1])

In [9]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([12500, 12500]))

In [10]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([12500, 12500]))

# Building and training the neural network

In [11]:
model_path = 'https://tfhub.dev/google/nnlm-en-dim50/2'

In [12]:
embedding_layer = hub.KerasLayer(model_path, input_shape = [], dtype = tf.string, trainable = True)

In [13]:
embedding_layer(X_train[0:2])

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [15]:
model = tf.keras.Sequential()
model.add(embedding_layer)
# 50 -> 16 -> 1
model.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 10, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 10)                170       
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 48,191,597
Trainable params: 48,191,597
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(optimizer = 'adam', loss = tf.losses.BinaryCrossentropy(from_logits = True), metrics = ['accuracy'])

In [17]:
model.fit(X_train, y_train, epochs = 50, batch_size = 512, verbose = 1)

Epoch 1/50
49/49 [==============================] - 5s 38ms/step - loss: 0.6182 - accuracy: 0.5686
Epoch 2/50
49/49 [==============================] - 2s 36ms/step - loss: 0.4061 - accuracy: 0.8144
Epoch 3/50
49/49 [==============================] - 2s 36ms/step - loss: 0.2552 - accuracy: 0.8954
Epoch 4/50
49/49 [==============================] - 2s 36ms/step - loss: 0.1730 - accuracy: 0.9352
Epoch 5/50
49/49 [==============================] - 2s 38ms/step - loss: 0.1164 - accuracy: 0.9606
Epoch 6/50
49/49 [==============================] - 2s 41ms/step - loss: 0.0763 - accuracy: 0.9771
Epoch 7/50
49/49 [==============================] - 2s 37ms/step - loss: 0.0475 - accuracy: 0.9890
Epoch 8/50
49/49 [==============================] - 2s 36ms/step - loss: 0.0294 - accuracy: 0.9949
Epoch 9/50
49/49 [==============================] - 2s 36ms/step - loss: 0.0182 - accuracy: 0.9987
Epoch 10/50
49/49 [==============================] - 2s 36ms/step - loss: 0.0118 - accuracy: 0.9993
Epoch 11/

In [18]:
results = model.evaluate(X_test, y_test)
print(results)

782/782 [==============================] - 4s 5ms/step - loss: 1.0529 - accuracy: 0.8458
[1.0529056787490845, 0.8458399772644043]


# Predictions

In [19]:
X_test[0:5]

array([b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
       b"A blackly comic tale of a down-trodden p

In [20]:
y_test[0:5]

array([1, 1, 0, 0, 1])

In [30]:
predictions = model.predict(X_test)

782/782 [==============================] - 3s 4ms/step


In [31]:
predictions[:15]

array([[  6.7941628 ],
       [ -0.16135532],
       [ -7.4415016 ],
       [-18.075293  ],
       [ 12.506494  ],
       [ 15.616564  ],
       [ 15.439349  ],
       [ 11.688524  ],
       [ 14.6345215 ],
       [  1.2759936 ],
       [-16.164955  ],
       [  1.3619974 ],
       [ 17.873018  ],
       [ -5.6601043 ],
       [  9.764158  ]], dtype=float32)

In [32]:
predictions = tf.nn.sigmoid(predictions).numpy()
predictions

array([[9.9888092e-01],
       [4.5974851e-01],
       [5.8606034e-04],
       ...,
       [4.9128852e-12],
       [9.9998713e-01],
       [9.9832219e-01]], dtype=float32)

In [36]:
y_test[0:15]

array([1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1])

In [34]:
predictions = (predictions >= 0.5)
predictions

array([[ True],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [ True]])

In [35]:
y_pred = predictions.astype(int)

In [37]:
y_pred[:15]

array([[1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1]])

Writing the Confusion Matrix

In [38]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[10716  1784]
 [ 2068 10432]]


0.84592

* TP = 10716
* FP = 1784
* FN = 2068
* TN = 10432

In [39]:
print('Accuracy: '+str(100*accuracy_score(y_test, y_pred)) +"%")

Accuracy: 84.592%
